## Notes

For notes refer to: [readme.md](readme.md) (with a markdown viewer of course)

## Libraries

In [1]:
from web3 import Web3
from web3.middleware import geth_poa_middleware

In [2]:
import os
import json
import requests

In [3]:
import numpy as np
import pandas as pd

In [4]:
from tqdm.notebook import tqdm, trange
import time

## Inits

### envs

### Web3

In [21]:
w3 = Web3(Web3.HTTPProvider("https://api.avax.network/ext/bc/C/rpc"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.isConnected()

True

### Addresses

In [22]:
cra = w3.toChecksumAddress("0xa32608e873f9ddef944b24798db69d80bbb4d1ed")
tus = w3.toChecksumAddress("0xf693248f96fe03422fea95ac0afbbbc4a8fdd172")
crabada=w3.toChecksumAddress("0x1b7966315ef0259de890f38f1bdb95acc03cacdd")
marketplace=w3.toChecksumAddress("0x7E8DEef5bb861cF158d8BdaAa1c31f7B49922F49")
game=w3.toChecksumAddress("0x82a85407BD612f52577909F4A58bfC6873f14DA8")

## Function Methods

### Crabada: 0x1b7966315ef0259de890f38f1bdb95acc03cacdd

- 0x1a46e42a: cancelSellOrder(uint256)
- 0xb88d4fde: safeTransferFrom(address,address,uint256,bytes)
- 0x598b8e71: deposit(uint256[] amounts)
- 0xc70f5eaa: buyCard(uint256)
- 0xd9ecad7b: breed(uint256,uint256)
- 0x8293744b: 

## Web3 Queries

### contract and abis

Should be atleast ERC721, ERC721Enumerable.

In [23]:
abi = json.loads(
'''
[{
"inputs": [], "name": "totalSupply", 
"outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}], 
"stateMutability": "view", 
"type": "function"
},
{
"inputs": [],
"name": "price",
"outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
"stateMutability": "view",
"type": "function"
},
{
"inputs": [{"internalType": "uint256", "name": "tokenId", "type": "uint256"}],
"name": "tokenURI", 
"outputs": [{"internalType": "string", "name": "", "type": "string"}],
"stateMutability": "view",
"type": "function"
},
{
"constant": true,
"inputs": [{"internalType": "uint256", "name": "tokenId", "type": "uint256"}],
"name": "ownerOf",
"outputs": [{"internalType": "address", "name": "", "type": "address"}],
"payable": false,
"stateMutability": "view",
"type": "function"
}
]
''')

In [24]:
crabContract = w3.eth.contract(address=crabada, abi=abi)

### contract calls

#### Sample Calls and Data

In [9]:
# total supply
crabContract.functions.totalSupply().call() #Findings

10018

In [10]:
# crab owner
crabContract.functions.ownerOf(1).call()

'0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d'

In [11]:
crabContract.functions.tokenURI(1).call()

'https://api.crabada.com/public/crabada/info-json/1'

{"description":"906392544231311161076231617881117286073401978432969551146651717139041419521",  
"external_url":"https://marketplace.crabada.com/crabada/1",   

"image":"https://pro-crabada-photos.s3.ap-southeast-1.amazonaws.com/1.png",  
"name":"Crabada 1",  
"attributes":[{"display_type":"date","trait_type":"birthday","value":1635777816},  
{"trait_type":"class","value":"SURGE"},  
{"trait_type":"pure","value":6},  
{"trait_type":"legend","value":0},  
{"trait_type":"origin","value":1},  
{"trait_type":"genesis","value":1},  
{"trait_type":"hp","value":150},  
{"trait_type":"speed","value":26},  
{"trait_type":"damage","value":50},  
{"trait_type":"armor","value":37},  
{"trait_type":"critical","value":39}]  
}

In [ ]:
#web3.eth.sendTransaction({'to': contractAddress, 'from': yourAddress, 'data': "0x61455567"})

#### Get Token to Owner List

In [95]:
# prep a data frame of all token numbers
df = pd.DataFrame(index=\
                  range(1,crabContract.functions.totalSupply().call()+1),
                  columns=["owner"]
                 )
print(df.size)

10003


In [96]:
# Call contract to get owners for all Tokens
for i in tqdm(df.index):
    df["owner"][i] = crabContract.functions.ownerOf(i).call()
    
    # status/heathcheck prints
    if i==1:print(df.loc[i])
    if i%1000==0:print(df.loc[i])

  0%|          | 0/10003 [00:00<?, ?it/s]

owner    0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
Name: 1, dtype: object
owner    0x1fA283b8C14e2d33e699cCe56Bf32b7cb2DB67d8
Name: 1000, dtype: object
owner    0x8D215C82d88835BF3F36e9E84E14a078fAA3afA7
Name: 2000, dtype: object
owner    0x7E8DEef5bb861cF158d8BdaAa1c31f7B49922F49
Name: 3000, dtype: object
owner    0x82a85407BD612f52577909F4A58bfC6873f14DA8
Name: 4000, dtype: object
owner    0x1fA283b8C14e2d33e699cCe56Bf32b7cb2DB67d8
Name: 5000, dtype: object
owner    0x1fA283b8C14e2d33e699cCe56Bf32b7cb2DB67d8
Name: 6000, dtype: object
owner    0x82a85407BD612f52577909F4A58bfC6873f14DA8
Name: 7000, dtype: object
owner    0x82a85407BD612f52577909F4A58bfC6873f14DA8
Name: 8000, dtype: object
owner    0x82a85407BD612f52577909F4A58bfC6873f14DA8
Name: 9000, dtype: object
owner    0x2E5322b59a2D5c982BE103fCcc05EB182AD9BcE3
Name: 10000, dtype: object


In [99]:
df.head()

,owner
1,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
2,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
3,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
4,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
5,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d


In [71]:
# save owner set
df.to_pickle("./data/raw/TokenToOwner.pkl")

In [9]:
# reload
df = pd.read_pickle("./data/raw/TokenToOwner.pkl")

In [68]:
# total supply
crabContract.functions.totalSupply().call()

10773

In [69]:
# Update owners df
for i in tqdm(range(df.index[-1]+1, 
               crabContract.functions.totalSupply().call()+1)):
    if i not in df.index:
        try:
            df = df.append(pd.DataFrame(index=[i], 
                                   columns=["owner"],
                                    data=[crabContract.functions.ownerOf(i).call()],
                                  ))
        except:
            pass
        

  0%|          | 0/1 [00:00<?, ?it/s]

## SnowTrace API Queries

In [42]:
apikey = os.getenv("SNOWTRACE_API")

### standard queries

In [75]:
q_balance = '''
https://api.snowtrace.io/api?module=account&action=tokenbalance&contractaddress={contractaddress}&address={address}&tag=latest&apikey={apitoken}
'''

### execute query

In [78]:
r = requests.get(q_balance
                 .format(
                        contractaddress=crabada,
                        address="0x1fa283b8c14e2d33e699cce56bf32b7cb2db67d8",
                        apitoken=apikey
                        )
                )
r.json()["result"]

'372'

## Crab API Queries

### NFT Token URIs

In [73]:
# crabada api
requests.get("https://api.crabada.com/public/crabada/info/"+"7881").json()

{'error_code': None,
 'message': None,
 'result': {'id': 7881,
  'crabada_id': 7881,
  'crabada_type': 1,
  'type': 'NORMAL',
  'is_origin': 0,
  'is_genesis': 0,
  'legend_number': 0,
  'pure_number': 3,
  'stage': 1,
  'dna': '593660613765537134739987960249627792532746782530721184354561233824127341390',
  'name': 'Crabada 7881',
  'description': None,
  'photo': '7881.png',
  'atlas_photo': '7881-atlas.png',
  'crabada_class': 6,
  'class_id': 6,
  'class_name': 'RUINED',
  'crabada_subclass': 80,
  'birthday': 1637528657,
  'hp': 118,
  'speed': 27,
  'damage': 62,
  'critical': 41,
  'armor': 25,
  'shell_id': 82,
  'shell_class': 6,
  'shell_name': 'S.Pharaoh',
  'shell_photo': 'shell_82.png',
  'horn_id': 83,
  'horn_class': 6,
  'horn_name': 'H.Crauldron',
  'horn_photo': 'horn_83.png',
  'body_id': 51,
  'body_class': 4,
  'body_name': 'B.Cropion',
  'body_photo': 'body_51.png',
  'mouth_id': 93,
  'mouth_class': 7,
  'mouth_name': 'M.Paraiba',
  'mouth_photo': 'mouth_93.png',


In [74]:
# contract tokenURI api
requests.get("https://api.crabada.com/public/crabada/info-json/"+"7881").json()

{'description': '593660613765537134739987960249627792532746782530721184354561233824127341390',
 'external_url': 'https://marketplace.crabada.com/crabada/7881',
 'image': 'https://photos.crabada.com/7881.png',
 'name': 'Crabada 7881',
 'attributes': [{'display_type': 'date',
   'trait_type': 'birthday',
   'value': 1637528657},
  {'trait_type': 'class', 'value': 'RUINED'},
  {'trait_type': 'pure', 'value': 3},
  {'trait_type': 'legend', 'value': 0},
  {'trait_type': 'origin', 'value': 0},
  {'trait_type': 'genesis', 'value': 0},
  {'trait_type': 'hp', 'value': 118},
  {'trait_type': 'speed', 'value': 27},
  {'trait_type': 'damage', 'value': 62},
  {'trait_type': 'armor', 'value': 25},
  {'trait_type': 'critical', 'value': 41}]}

In [128]:
dfURI = pd.DataFrame(index=range(1, df.size+1), columns=["data"])

for i in tqdm(dfURI.index):
    dfURI["data"][i]= requests.get("https://api.crabada.com/public/crabada/info-json/"+str(i)).json()
    
    # status/heathcheck prints
    if i==1:print(dfURI.loc[i])
    if i%1000==0:print(dfURI.loc[i])

  0%|          | 0/10003 [00:00<?, ?it/s]

data    {'description': '90639254423131116107623161788...
Name: 1, dtype: object
data    {'description': '65019971983844543328665348027...
Name: 1000, dtype: object
data    {'description': '64313233157933189596832029027...
Name: 2000, dtype: object
data    {'description': '53535466062785045186373914272...
Name: 3000, dtype: object
data    {'description': '54242204888696398918207233272...
Name: 4000, dtype: object
data    {'description': '57069160192341813845540509273...
Name: 5000, dtype: object
data    {'description': '51591934291528822423832287021...
Name: 6000, dtype: object
data    {'description': '54065520182218560485248903522...
Name: 7000, dtype: object
data    {'description': '54065520182218560485248903522...
Name: 8000, dtype: object
data    {'description': '59896115495987228772873785275...
Name: 9000, dtype: object
data    {'description': None, 'external_url': 'https:/...
Name: 10000, dtype: object


In [23]:
# Expand dictionary of "data" into columns
dfURI["attributes"] =dfURI["data"].apply(lambda x: x["attributes"])
dfURI = pd.concat([dfURI, dfURI["data"].apply(pd.Series).drop(["attributes"], axis=1)], axis=1)

In [33]:
# create function to extract traits from attributes
def extract_attr(a):
    traits = dict()
    for d in a:
        traits.update({d["trait_type"]: d["value"]})
    return traits

In [34]:
# extract then concat traits
dfURI["traits"] = dfURI["attributes"].apply(lambda x: extract_attr(x))
dfURI = pd.concat([dfURI, dfURI["traits"].apply(pd.Series)], axis=1)

# some are without stats and traits yet
#dfURI[dfURI["class"].isna()]

In [44]:
dfURI.to_pickle("./data/raw/tokenMeta.pkl")

### Get NFT Crab Data via Crabada Specified API (most advisable)

In [25]:
dfCrabData = pd.DataFrame(columns=["data"])

In [26]:
# Update owners df
try: 
    startindex = dfCrabData.index[-1]
except:
    startindex = 0
    
for i in tqdm(range(startindex+1, crabContract.functions.totalSupply().call()+1)):
    if i not in dfCrabData.index:
        try:
            dfCrabData = dfCrabData.append(pd.DataFrame(index=[i], 
                                   columns=["data"],
                                    data=[[requests.get("https://api.crabada.com/public/crabada/info/"+str(i)).json()]],
                                  ))
        except:
            pass

  0%|          | 0/10772 [00:00<?, ?it/s]

In [45]:
dfCrabData.to_pickle("./data/raw/crabAPIdata.pkl")